In [14]:
import pandas as pd
import datetime
import os
import sqlite3
from sklearn.preprocessing import LabelEncoder

In [19]:
#parameters
date = ''
airflow_home = os.environ['AIRFLOW_HOME']

In [20]:
data = pd.read_csv(f'{airflow_home}/data/monthly/output_{date}.csv')

In [21]:
order_cnt = data[['OrderID', 'OrderCnt']].groupby('OrderID').sum().reset_index()

In [22]:
data = data.drop(['ClientID', 'MaterialID', 'GroupID', 'OrderCnt'], axis=1)

In [23]:
interval_time = data['Interval'].apply(lambda x: int(x.split('-')[1][:-1]) - int(x.split('-')[0]))

In [24]:
order_weekday = data['OrderDate'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').weekday())
weekday = data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').weekday())

In [25]:
interval_low = data['Interval'].apply(lambda x: int(x.split('-')[0])).apply(lambda x: x if x != 0 else 24)
interval_high = data['Interval'].apply(lambda x: int(x.split('-')[1][:-1])).apply(lambda x: x if x != 0 else 24)
interval_time = interval_high - interval_low
interval_avg = (interval_high + interval_low) / 2

In [26]:
morning = list(range(6, 12))
day = list(range(12, 18))
evening = list(range(18, 24))
night = [24] + list(range(1, 6))

morning = interval_avg.isin(morning)
day = interval_avg.isin(day)
evening = interval_avg.isin(evening)
night = interval_avg.isin(night)

In [27]:
data['interval_time'] = interval_time
data['order_weekday'] = order_weekday
data['weekday'] = weekday
data['interval_low'] = interval_low
data['interval_high'] = interval_high
data['interval_time'] = interval_time
data['interval_avg'] = interval_avg
data['morning'] = morning.apply(lambda x: 1 if x else 0)
data['day'] = day.apply(lambda x: 1 if x else 0)
data['evening'] = evening.apply(lambda x: 1 if x else 0)
data['night'] = night.apply(lambda x: 1 if x else 0)
data['is_order_weekend'] = data['order_weekday'].apply(lambda x: 1 if x in [5,6] else 0)
data['is_weekend'] = data['weekday'].apply(lambda x: 1 if x in [5,6] else 0)
data['Cluster'] = data['Cluster'].fillna('foo')
data['count_edit'] -= 1

In [28]:
label_cols = ['DeliveryType', 'Cluster']

for col in label_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

In [35]:
data = data.drop(['index', 'Interval', 'Date'], axis=1)

In [36]:
conn = sqlite3.connect(f'{airflow_home}/db/sqlite.db')
if_exists = 'append'
if date == '2018-02-01':
    if_exists = 'replace'
data.to_sql('orders_preprocessed', conn, if_exists=if_exists)

In [40]:
data = pd.read_sql('SELECT * FROM orders_preprocessed', conn)
data = data.drop('index', axis=1)
data.to_csv(f'{airflow_home}/data/data_preprocessed.csv', index=False)